In [1]:
import os
import sys
import numpy as np
import copy                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
import scipy

from sklearn.preprocessing import normalize

sys.path.insert(0, "/home/anonymous/Desktop/eeg_sampling/modeling")
from dataio import readdata2, readlabels2, writedata
from datafilters import apply_dc_filter, apply_dwt_filter
from wignerville import wvd, filtered_wvd

In [2]:
dataset = readdata2("./../curated/raw-samples")
labels = readlabels2("./../curated/raw-inputs")

#Constants
fs = 250.0 #Frequency in Hz
sample_time = dataset[0].shape[1]/fs #Total time for sample
num_rows = 480 
num_channels = 8
rto = num_rows/(fs/2)

In [3]:
master_dataset = []

# Channel 1 (For right hand movement)
data = []
for i in range(0, len(dataset)):
    data.append(np.delete(dataset[i], [0,2,3,4,5,6,7], 2))
    data[i].shape = (data[i].shape[0], data[i].shape[1])
master_dataset.append(data)

# Channel 3 (For left hand movement)
data = []
for i in range(0, len(dataset)):
    data.append(np.delete(dataset[i], [0,1,2,4,5,6,7], 2))
    data[i].shape = (data[i].shape[0], data[i].shape[1])
master_dataset.append(data)
  
# Grand Average
data = []
for i in range(0,len(dataset)):
    data.append(np.mean(dataset[i],axis=2))
master_dataset.append(data)

In [4]:
#First filter
dc_lowcut = 1.0
dc_highcut = 13.0
dc_order = 2
dc_type = "bandpass"
dc_func_type = "butter"
for i in range(0, len(master_dataset)):
    for j in range(0, len(master_dataset[i])):
        for k in range(0, master_dataset[i][j].shape[0]):
            master_dataset[i][j][k] = apply_dc_filter(master_dataset[i][j][k], fs, dc_lowcut, dc_highcut, dc_order, dc_type, dc_func_type)

In [5]:
#Wigner-Ville Transform with Normalization and Cut
norm="l1"

freq_min = 1.0
freq_max = 13.0
del_values = np.concatenate([np.arange(0, int(freq_min*rto)), np.arange(int(freq_max*rto), 480)])

master_wv = []
for i in range(0, len(master_dataset)):
    trial = []
    for j in range(0, len(master_dataset[i])):
        sample = []
        for k in range(0, master_dataset[i][j].shape[0]):
            wv_tmp = wvd(master_dataset[i][j][k])[0] #Compute Wigner-Ville
            #wv_tmp = normalize(wv_tmp, axis=1, norm=norm) #Normalize data
            wv_tmp = wv_tmp.T #Get Transform
            wv_tmp = np.delete(wv_tmp, del_values, 0) #Cut to important data
            wv_tmp = scipy.absolute(wv_tmp) #Get rid of neg values
            sample.append(wv_tmp)
        trial.append(sample)
    master_wv.append(trial)

In [ ]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
#Test
extent = [0, sample_time, freq_min, freq_max]
interpolation = "lanczos"


plt.figure(1)
plt.figure(figsize=(8,3))
plt.imshow(master_wv[0][0][0], extent=extent, origin='lower', aspect='auto', interpolation=interpolation)
plt.colorbar()

In [6]:
# Test Set
test_index = 4 #Last one
test_dataset = []
train_dataset = []
for i in range(0, len(master_wv)):
    test_dataset.append(master_wv[i][test_index])
    master_wv[i] = np.delete(master_wv[i], test_index, axis = 0)
    train_dataset.append(master_wv[i])
    
test_labels = labels[test_index]
labels = np.delete(labels, test_index, axis = 0)
train_labels = labels


# Train Set
for i in range(0, len(train_dataset)):
    train_dataset[i] = np.concatenate(train_dataset[i])

train_labels = np.concatenate(labels)

In [7]:
train_right = train_dataset[0].astype(np.float32)
train_left = train_dataset[1].astype(np.float32)
train_avg = train_dataset[2].astype(np.float32)

test_right = np.array(test_dataset[0], dtype=np.float32)
test_left = np.array(test_dataset[1], dtype=np.float32)
test_avg = np.array(test_dataset[2], dtype=np.float32)

#Write training
writedata("./../curated/train_dataset_right", train_right)
writedata("./../curated/train_dataset_left", train_left)
writedata("./../curated/train_dataset_avg", train_avg)
writedata("./../curated/train_labels", train_labels.astype(np.uint8))

#Write test
writedata("./../curated/test_dataset_right", test_right)
writedata("./../curated/test_dataset_left", test_left)
writedata("./../curated/test_dataset_avg", test_avg)
writedata("./../curated/test_labels", test_labels.astype(np.int8))

In [ ]:
train_right.shape